#AIDM7360 Group Project
**Group Members:ZHU Xiaoyu, LUO Fan, LUO jinjin**

In this SQL project we hope to address the following questions by further processing and analysing the Hong Kong Airbnb data in 2023.



In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive') # do not change!

Mounted at /content/drive


In [ ]:
import os, pathlib
dataPath = '/content/drive/MyDrive/Colab Notebooks/data/' # the file path
if not(os.path.exists(dataPath)):
    path = pathlib.Path(dataPath)
    path.mkdir(parents=True, exist_ok=True) # Can create the folders in the path if missing. No error if path exists
else:
    print('The data path you selected already exists')

The data path you selected already exists


In [ ]:
# set up
import sqlite3
from sqlite3 import Error

import pandas as pd
import numpy as np

In [ ]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=210030dc979e34497662292123c260f83e083563bbb6e6340b5bd908b75e008e
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
# download the dataset
import wget

url = 'https://raw.githubusercontent.com/JennyLUO77/jennyluo77.github.io/AIDM7360/AB_HK_2023.csv'
fileName = wget.download(url, out=dataPath)
airbnb_df = pd.read_csv(fileName)
airbnb_df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,17891,Rental unit in Hong Kong Island · ★4.76 · Stud...,69063,Candace,NaN,Central & Western,22.283270,114.149880,Entire home/apt,1400,60,73,2017-11-29,0.45,1,0,0,NaN
1,72571,Rental unit in Sheung Wan · ★4.22 · Studio · 1...,304876,Brendan,NaN,Central & Western,22.284630,114.150540,Entire home/apt,368,28,151,2022-03-11,0.99,10,0,0,NaN
2,103760,Rental unit in Central · ★4.46 · 2 bedrooms · ...,304876,Brendan,NaN,Central & Western,22.284180,114.154310,Entire home/apt,1046,4,304,2023-08-29,2.04,10,299,20,NaN
3,104626,Rental unit in Hong Kong · ★4.38 · 1 bedroom ·...,544166,Celine,NaN,Central & Western,22.283600,114.147900,Entire home/apt,2336,1,15,2023-04-28,0.10,1,365,1,NaN
4,132773,Rental unit in Hong Kong Island · ★4.50 · 2 be...,304876,Brendan,NaN,Central & Western,22.289210,114.143250,Entire home/apt,1166,5,239,2023-09-01,1.61,10,284,23,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6730,980852429746950881,Rental unit in Hong Kong · ★New · Studio · 2 b...,6735446,Eunice,NaN,Wan Chai,22.279171,114.191708,Entire home/apt,950,1,0,NaN,NaN,1,153,0,NaN
6731,980885923625264869,Rental unit in Hong Kong · ★New · Studio · 1 b...,54921788,Benjamin,NaN,Central & Western,22.288150,114.143320,Entire home/apt,790,28,0,NaN,NaN,2,269,0,NaN
6732,981023398549117903,kezhan in Hong Kong · ★New · 1 bedroom · 1 bed...,529558920,晓敏,NaN,Yau Tsim Mong,22.295870,114.170740,Private room,414,1,0,NaN,NaN,2,270,0,NaN
6733,981185183376149872,Rental unit in Hong Kong · ★New · 1 bedroom · ...,6763017,Rina,NaN,Southern,22.255838,114.201043,Entire home/apt,2380,16,0,NaN,NaN,7,180,0,NaN


#Data Cleaning
This data contains empty columns. And for this data the name of the Airbnb incrould the room descriptions, so a same hotel may display multiple times which is not good for our project.

**Cleaning plan**

1. Remove the empty columns.
2. Remove the duplicate hotel (Using the latitude and longitude. Because they have unique id but with the same location.)
3. Clean the name column. Remove the stars and room descriptions.

In [ ]:
# Check the number of rows and columes
airbnb_df.shape

(6735, 18)

In [ ]:
# Check the data types
airbnb_df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group               float64
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
number_of_reviews_ltm               int64
license                           float64
dtype: object

In [ ]:
# Chack the missing value
airbnb_df.isna().sum()

id                                   0
name                                 0
host_id                              0
host_name                            0
neighbourhood_group               6735
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                       3452
reviews_per_month                 3452
calculated_host_listings_count       0
availability_365                     0
number_of_reviews_ltm                0
license                           6735
dtype: int64

In [ ]:
# Delet the empty columns
airbnb_df.dropna(axis=1, how ='all', inplace=True)
airbnb_df

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,17891,Rental unit in Hong Kong Island · ★4.76 · Stud...,69063,Candace,Central & Western,22.283270,114.149880,Entire home/apt,1400,60,73,2017-11-29,0.45,1,0,0
1,72571,Rental unit in Sheung Wan · ★4.22 · Studio · 1...,304876,Brendan,Central & Western,22.284630,114.150540,Entire home/apt,368,28,151,2022-03-11,0.99,10,0,0
2,103760,Rental unit in Central · ★4.46 · 2 bedrooms · ...,304876,Brendan,Central & Western,22.284180,114.154310,Entire home/apt,1046,4,304,2023-08-29,2.04,10,299,20
3,104626,Rental unit in Hong Kong · ★4.38 · 1 bedroom ·...,544166,Celine,Central & Western,22.283600,114.147900,Entire home/apt,2336,1,15,2023-04-28,0.10,1,365,1
4,132773,Rental unit in Hong Kong Island · ★4.50 · 2 be...,304876,Brendan,Central & Western,22.289210,114.143250,Entire home/apt,1166,5,239,2023-09-01,1.61,10,284,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6730,980852429746950881,Rental unit in Hong Kong · ★New · Studio · 2 b...,6735446,Eunice,Wan Chai,22.279171,114.191708,Entire home/apt,950,1,0,NaN,NaN,1,153,0
6731,980885923625264869,Rental unit in Hong Kong · ★New · Studio · 1 b...,54921788,Benjamin,Central & Western,22.288150,114.143320,Entire home/apt,790,28,0,NaN,NaN,2,269,0
6732,981023398549117903,kezhan in Hong Kong · ★New · 1 bedroom · 1 bed...,529558920,晓敏,Yau Tsim Mong,22.295870,114.170740,Private room,414,1,0,NaN,NaN,2,270,0
6733,981185183376149872,Rental unit in Hong Kong · ★New · 1 bedroom · ...,6763017,Rina,Southern,22.255838,114.201043,Entire home/apt,2380,16,0,NaN,NaN,7,180,0


In [ ]:
# Fill the 'last_review'(str) and 'review_per_month'(int)
airbnb_df.fillna({'last_review': 'no record'}, inplace=True)
airbnb_df.fillna({'reviews_per_month': '0'}, inplace=True)
airbnb_df

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,17891,Rental unit in Hong Kong Island · ★4.76 · Stud...,69063,Candace,Central & Western,22.283270,114.149880,Entire home/apt,1400,60,73,2017-11-29,0.45,1,0,0
1,72571,Rental unit in Sheung Wan · ★4.22 · Studio · 1...,304876,Brendan,Central & Western,22.284630,114.150540,Entire home/apt,368,28,151,2022-03-11,0.99,10,0,0
2,103760,Rental unit in Central · ★4.46 · 2 bedrooms · ...,304876,Brendan,Central & Western,22.284180,114.154310,Entire home/apt,1046,4,304,2023-08-29,2.04,10,299,20
3,104626,Rental unit in Hong Kong · ★4.38 · 1 bedroom ·...,544166,Celine,Central & Western,22.283600,114.147900,Entire home/apt,2336,1,15,2023-04-28,0.1,1,365,1
4,132773,Rental unit in Hong Kong Island · ★4.50 · 2 be...,304876,Brendan,Central & Western,22.289210,114.143250,Entire home/apt,1166,5,239,2023-09-01,1.61,10,284,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6730,980852429746950881,Rental unit in Hong Kong · ★New · Studio · 2 b...,6735446,Eunice,Wan Chai,22.279171,114.191708,Entire home/apt,950,1,0,no record,0,1,153,0
6731,980885923625264869,Rental unit in Hong Kong · ★New · Studio · 1 b...,54921788,Benjamin,Central & Western,22.288150,114.143320,Entire home/apt,790,28,0,no record,0,2,269,0
6732,981023398549117903,kezhan in Hong Kong · ★New · 1 bedroom · 1 bed...,529558920,晓敏,Yau Tsim Mong,22.295870,114.170740,Private room,414,1,0,no record,0,2,270,0
6733,981185183376149872,Rental unit in Hong Kong · ★New · 1 bedroom · ...,6763017,Rina,Southern,22.255838,114.201043,Entire home/apt,2380,16,0,no record,0,7,180,0


In [ ]:
# chack duplicated location (latitude, longitude)
airbnb_df.duplicated(subset=['latitude','longitude'])
duplicate_df = airbnb_df[airbnb_df.duplicated(subset=['latitude','longitude'])]
print('duplicated rows:', duplicate_df.shape[0])

duplicated rows: 313


In [ ]:
# Remove duplicates
airbnb_df.drop_duplicates(subset=['latitude','longitude'], inplace=True)

# How many left?
print(airbnb_df.shape[0], 'rows left after removing duplicates')

6422 rows left after removing duplicates


In [ ]:
# Clean the name column
name = airbnb_df['name']
# Split the string and select the first item
name = name.str.split(' ·',).str[0]
name

0       Rental unit in Hong Kong Island
1             Rental unit in Sheung Wan
2                Rental unit in Central
3              Rental unit in Hong Kong
4       Rental unit in Hong Kong Island
                     ...               
6730           Rental unit in Hong Kong
6731           Rental unit in Hong Kong
6732                kezhan in Hong Kong
6733           Rental unit in Hong Kong
6734            Guesthouse in Hong Kong
Name: name, Length: 6422, dtype: object

In [ ]:
# Replace the original one
airbnb_df['name'] = name
airbnb_df

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,17891,Rental unit in Hong Kong Island,69063,Candace,Central & Western,22.283270,114.149880,Entire home/apt,1400,60,73,2017-11-29,0.45,1,0,0
1,72571,Rental unit in Sheung Wan,304876,Brendan,Central & Western,22.284630,114.150540,Entire home/apt,368,28,151,2022-03-11,0.99,10,0,0
2,103760,Rental unit in Central,304876,Brendan,Central & Western,22.284180,114.154310,Entire home/apt,1046,4,304,2023-08-29,2.04,10,299,20
3,104626,Rental unit in Hong Kong,544166,Celine,Central & Western,22.283600,114.147900,Entire home/apt,2336,1,15,2023-04-28,0.1,1,365,1
4,132773,Rental unit in Hong Kong Island,304876,Brendan,Central & Western,22.289210,114.143250,Entire home/apt,1166,5,239,2023-09-01,1.61,10,284,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6730,980852429746950881,Rental unit in Hong Kong,6735446,Eunice,Wan Chai,22.279171,114.191708,Entire home/apt,950,1,0,no record,0,1,153,0
6731,980885923625264869,Rental unit in Hong Kong,54921788,Benjamin,Central & Western,22.288150,114.143320,Entire home/apt,790,28,0,no record,0,2,269,0
6732,981023398549117903,kezhan in Hong Kong,529558920,晓敏,Yau Tsim Mong,22.295870,114.170740,Private room,414,1,0,no record,0,2,270,0
6733,981185183376149872,Rental unit in Hong Kong,6763017,Rina,Southern,22.255838,114.201043,Entire home/apt,2380,16,0,no record,0,7,180,0


After cleaning the databace, we found out this data actually have a lot of airbnb with same name but with different host and location. So we may take those are different.

#Create a connection with SQLite

In [ ]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

# Design the relations
**Tables:**

*Airbnb* (`id`, name, room_type, price, minimun_night, availability_365)

*Host* (`id`, `host_id`, host_name)

*Location* (`id`,  neighbourhood, latitude, longitude)

*Review* (`id`, number_of_reviews, last_review, reviews_per_month)



In [ ]:
# Select the values we need in the csv file
# values for airbnb table
airbnb_values = airbnb_df[['id', 'name', 'room_type', 'minimum_nights', 'price']].values
# values for host table
host_values = airbnb_df[['id', 'host_id', 'host_name']].values[:]
# values for location table
location_values = airbnb_df[['id', 'neighbourhood', 'latitude','longitude']].values
# values for review table
review_values = airbnb_df[['id', 'number_of_reviews', 'last_review','reviews_per_month']].values

##Airbnb table setup

In [ ]:
# Create aiebnb table
database = dataPath + 'Airbnb.db'
conn = create_connection(database)
#Creating a cursor object using the cursor() method
cur = conn.cursor()

#Drop AIRBNB table if already exists. (if our databasc have table "AIRBNB", drop it. if not, do nothing.)
cur.execute("DROP TABLE IF EXISTS AIRBNB")

#Creating table as per requirement
sql ='''CREATE TABLE AIRBNB
    (ID INT,
    NAME,
    ROOM_TYPE,
    MINIMUN_NIGHT INT,
    PRICE INT,
    PRIMARY KEY(Id))'''

cur.execute(sql)


In [ ]:
# Defined the insert funtion
def addAirbnb(conn, values):
    """
    Insert data into database
    A sample input is: 17891, Rental unit in Hong Kong Island, Entire home/apt, 60, 1400
    :return: lastrow id
    """
    with conn:
        sql = "INSERT INTO AIRBNB VALUES(?, ?, ?, ?, ?)" # the question mark (?) as the placeholder占位符
        cur = conn.cursor()
        cur.execute(sql, (values[0], values[1], values[2], values[3], values[4]))
        return cur.lastrowid

In [ ]:
# Insert the value in to airbnb data
database = dataPath + 'Airbnb.db'
conn = create_connection(database)
all_values = airbnb_values
for values in all_values:
    addAirbnb(conn, values)

##Host table setup

In [ ]:
conn = create_connection(database)
#Creating a cursor object using the cursor() method
cur = conn.cursor()

#Drop HOST table if already exists. (if our databasc have table "HOST", drop it. if not, do nothing.)
cur.execute("DROP TABLE IF EXISTS HOST")

#Creating table as per requirement
sql ='''CREATE TABLE HOST
    (ID INT,
    HOST_ID,
    HOST_NAME,
    PRIMARY KEY(Id))'''

cur.execute(sql)

In [ ]:
# Defined the insert funtion
def addHost(conn, values):
    """
    Insert data into database
    A sample input is: 17891, 69063, Candace
    :return: lastrow id
    """
    with conn:
        sql = "INSERT INTO HOST VALUES(?, ?, ?)" # the question mark (?) as the placeholder占位符
        cur = conn.cursor()
        cur.execute(sql, (values[0], values[1], values[2]))
        return cur.lastrowid

In [ ]:
# Insert the value in to airbnb data
database = dataPath + 'Airbnb.db'
conn = create_connection(database)
all_values = host_values
for values in all_values:
    addHost(conn, values)

##Location table setup

In [ ]:
conn = create_connection(database)
#Creating a cursor object using the cursor() method
cur = conn.cursor()

#Drop LOCATION table if already exists. (if our databasc have table "LOCATION", drop it. if not, do nothing.)
cur.execute("DROP TABLE IF EXISTS LOCATION")

#Creating table as per requirement
sql ='''CREATE TABLE LOCATION
    (ID INT,
    NEIGHBOURHOOD STR,
    LATITUDE,
    LONGITUDE,
    PRIMARY KEY(Id))'''

cur.execute(sql)

In [ ]:
# Defined the insert funtion
def addLocation(conn, values):
    """
    Insert data into database
    A sample input is: id, neighberhood, latitude, longitude
    :return: lastrow id
    """
    with conn:
        sql = "INSERT INTO LOCATION VALUES(?, ?, ?,?)" # the question mark (?) as the placeholder占位符
        cur = conn.cursor()
        cur.execute(sql, (values[0], values[1], values[2], values[3]))
        return cur.lastrowid

In [ ]:
# Insert the value in to airbnb data
database = dataPath + 'Airbnb.db'
conn = create_connection(database)
all_values = location_values
for values in all_values:
    addLocation(conn, values)

##Review table setup

In [ ]:
conn = create_connection(database)
#Creating a cursor object using the cursor() method
cur = conn.cursor()

#Drop REVIEW table if already exists. (if our databasc have table "REVIEW", drop it. if not, do nothing.)
cur.execute("DROP TABLE IF EXISTS REVIEW")

#Creating table as per requirement
sql ='''CREATE TABLE REVIEW
    (ID INT,
    NUMBER_OF_REVIEWS INT,
    LAST_REVIEW,
    REVIEWS_PER_MONTH,
    PRIMARY KEY(Id))'''

cur.execute(sql)

In [ ]:
# Defined the insert funtion
def addReview(conn, values):
    """
    Insert data into database
    A sample input is: id, number_of_reviews, last_review, reviews_per_month
    :return: lastrow id
    """
    with conn:
        sql = "INSERT INTO REVIEW VALUES(?, ?, ?, ?)" # the question mark (?) as the placeholder占位符
        cur = conn.cursor()
        cur.execute(sql, (values[0], values[1], values[2], values[3],))
        return cur.lastrowid

In [ ]:
# Insert the value in to airbnb data
database = dataPath + 'Airbnb.db'
conn = create_connection(database)
all_values = review_values
for values in review_values:
    addReview(conn, values)

##Template table setup

In [ ]:
# download the dataset
import wget

url = 'https://raw.githubusercontent.com/JennyLUO77/jennyluo77.github.io/main/templates.csv'
fileName = wget.download(url, out=dataPath)
template_df = pd.read_csv(fileName)
template_df

,tid,text
0,1,"Based on the survey, it can be found that the ..."
1,2,What is concerned the most is that the price o...
2,3,"This airbnb listing is owned by {host_name},wh..."
3,4,"For this room type, its average minimum night ..."
4,5,"For this popular host,you can know that the li..."
5,6,"The Most expensive room name is {name}, the di..."
6,7,The listing {name} has just received the revie...


In [ ]:
# update the path
template_df = pd.read_csv(dataPath + 'templates.csv')
template_df

,tid,text
0,1,"Based on the survey, it can be found that the ..."
1,2,What is concerned the most is that the price o...
2,3,"This airbnb listing is owned by {host_name},wh..."
3,4,"For this room type, its average minimum night ..."
4,5,"For this popular host,you can know that the li..."
5,6,"The Most expensive room name is {name}, the di..."
6,7,The listing {name} has just received the revie...


In [ ]:
# get the value of template
template_values = template_df[['tid', 'text']].values

In [ ]:
database = dataPath + 'Airbnb.db'
conn = create_connection(database)
#Creating a cursor object using the cursor() method
cur = conn.cursor()

#Drop LOCATION table if already exists. (if our databasc have table "LOCATION", drop it. if not, do nothing.)
cur.execute("DROP TABLE IF EXISTS TEMPLATE")

#Creating table as per requirement
sql ='''CREATE TABLE TEMPLATE
    (TID INT,
    TEXT STR,
    PRIMARY KEY(TID))'''

cur.execute(sql)

In [ ]:
# Defined the insert funtion
def addTemplate(conn, values):
    """
    Insert data into database
    A sample input is: tid, template.
    :return: lastrow tid
    """
    with conn:
        sql = "INSERT INTO TEMPLATE VALUES(?, ?)" # the question mark (?) as the placeholder占位符
        cur = conn.cursor()
        cur.execute(sql, (values[0], values[1]))
        return cur.lastrowid

In [ ]:
# Insert the value in to airbnb data
database = dataPath + 'Airbnb.db'
conn = create_connection(database)
all_values = template_values
for values in template_values:
    addTemplate(conn, values)

Data cleaning and database setup were finished.

#run this in case you haven't connect to SQLite

In [ ]:
# run this in case you haven't connect to SQLite
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

#Query

Select specific property information by ID

In [ ]:
def printRoomInfo(id, conn):
    """
    Print out the infomation of a room given a id
    """
    with conn:
        sql = "SELECT * FROM AIRBNB WHERE ID = ?"
        cur = conn.cursor()
        cur.execute(sql, [id])
        result = cur.fetchone()
        if result is None:
            print("No such rooms")
        else:
            headers = ["id", "name", "room_type", "minimum_night", "price"]
            for i in range(0, 5):
                print(headers[i], ":", result[i])

In [ ]:
def printRoomByNo(conn):
    """
    Select out a room according to the id.
    """
    listALLRooms(conn)
    line = input("Please input the id to print info:")
    line = line.strip()
    printRoomInfo(line, conn)

In [ ]:
def listMostPopularRoomType(conn):

    with conn:
            sql = """
                SELECT A.room_type, SUM(R.number_of_reviews) as total_reviews
                FROM Airbnb A
                JOIN Review R ON A.id = R.id
                GROUP BY A.room_type
                ORDER BY total_reviews DESC
                LIMIT 1 """
            cur = conn.cursor()
            cur.execute(sql)
            result_set = cur.fetchall()
            if result_set:
                for result in result_set:
                  room_type = result[0]
                  number_of_reviews = result[1]
                  print("The most popular room type is", room_type, "with total", number_of_reviews,"reviews")
            else:
                print ("NO such room type")


In [ ]:
listMostPopularRoomType(conn)

The most popular room type is Entire home/apt with total 42715 reviews


Query for template tid:1

Choose the most popular listing

In [ ]:
def listMost_PopularRoomType(conn):

    with conn:
            sql = "SELECT TEXT FROM TEMPLATE WHERE TID= 1"
            cur = conn.cursor()
            cur.execute(sql)
            result = cur.fetchone()
            template = result[0]
            # print(template)

            sql = """
                SELECT NAME,MAX(NUMBER_OF_REVIEWS) AS MAX_NUMBER_REVIEWS,A.ID
                FROM Airbnb AS A,REVIEW AS R
                WHERE A.ID = R.ID
                  """
            cur = conn.cursor()
            cur.execute(sql)
            result_set = cur.fetchall()
            if result_set:
                for result in result_set:

                  print(template.replace("{name}",str(result[0])).replace("{number_of_reviews}",str(result[1])).replace("{id}",str(result[2])))
            else:
                print ("NO such room type")


In [ ]:
listMost_PopularRoomType(conn)

Based on the survey, it can be found that the most popular Airbnb listing in Hong Kong in 2023 was the Rental unit in Hong Kong, which received the most 501 items of  reviews,and you can search it by id:850531


Query for template tid:2

In [ ]:
def printPrice_Roomtype(conn):
    """
    Print out the infomation of a listing's room type and price
    """
    line=input("please print the id:")
    with conn:
        sql = "SELECT TEXT FROM TEMPLATE WHERE TID= 2"
        cur = conn.cursor()
        cur.execute(sql)
        result = cur.fetchone()
        template = result[0]
        # print(template)

        sql = "SELECT PRICE,ROOM_TYPE FROM AIRBNB WHERE ID= ?"
        cur = conn.cursor()
        cur.execute(sql,(line,))
        result = cur.fetchone()

        if result is None:
            print("No such rooms")
        else:
            print(template.replace("{price}",str(result[0])).replace("{room_type}",str(result[1])))

In [ ]:
database = dataPath + 'Airbnb.db'
conn = create_connection(database)
printPrice_Roomtype(conn)

please print the id:17891
What is concerned the most is that the price of this listing is HK$ 1400,with a Entire home/apt room type.


Query for template tid:3

In [ ]:
def printNeighbour_host(conn):
    """
    Print out the infomation of a room given a neighbourhood
    """
    line=input("please print the id:")
    with conn:
        sql = "SELECT TEXT FROM TEMPLATE WHERE TID= 3"
        cur = conn.cursor()
        cur.execute(sql)
        result = cur.fetchone()
        template = result[0]
        #print(template)

        sql = "SELECT HOST_NAME,NEIGHBOURHOOD FROM HOST AS H,LOCATION AS L WHERE H.ID = L.ID AND H.ID= ?"
        cur = conn.cursor()
        cur.execute(sql,(line,))
        result = cur.fetchone()
        if result is None:
            print("No such rooms")
        else:
            print(template.replace("{host_name}",str(result[0])).replace("{neighbourhood}",str(result[1])))

In [ ]:
database = dataPath + 'Airbnb.db'
conn = create_connection(database)
printNeighbour_host(conn)

please print the id:17891
This airbnb listing is owned by Candace,which is located in community Central & Western.


Query for template tid:4

Compute the average minimum night by selecting certain room type

In [ ]:
def printMinimumNight(conn):
    """
    Print out the average minimum night of certain  room type

    """
    line=input("please print the room type:")
    with conn:
        sql = "SELECT TEXT FROM TEMPLATE WHERE TID= 4"
        cur = conn.cursor()
        cur.execute(sql)
        result = cur.fetchone()
        template = result[0]
        #print(template)

        sql = "SELECT FLOOR(AVG(MINIMUN_NIGHT)) AS AVERAGE_NIGHT FROM AIRBNB WHERE ROOM_TYPE =?"
        cur = conn.cursor()
        cur.execute(sql, (line,))
        result = cur.fetchone()
        if result is None:
            print("No such record")
        else:
            print(template.replace("{avg(minimum_night)}",str(result[0])))

In [ ]:
database = dataPath + 'Airbnb.db'
conn = create_connection(database)
printMinimumNight(conn)

please print the room type:Private room
For this room type, its average minimum night that the customers need to book is 21.0 nights.


Query for template tid:5

Search the Airbnb listing by use filter condition in host name

In [ ]:
def selectListing_host(conn):
    """
    Given the hostname(like some popular hosts), select all the listings filtered by this condition.
    """
    line = input("Please input host name: ")


    with conn:
        sql = "SELECT TEXT FROM TEMPLATE WHERE TID= 5"
        cur = conn.cursor()
        cur.execute(sql)
        result = cur.fetchone()
        template = result[0]
        #print(template)

        sql = "SELECT NAME,ROOM_TYPE,PRICE FROM AIRBNB WHERE ID=(SELECT ID FROM HOST WHERE HOST_NAME =?) "
        cur = conn.cursor()
        cur.execute(sql, (line,))
        result = cur.fetchone()
        if result is None:
            print("No such rooms")
        else:
            print(template.replace("{name}",str(result[0])).replace("{room_type}",str(result[1])).replace("{price}",str(result[2])))

In [ ]:
database = dataPath + 'Airbnb.db'
conn = create_connection(database)
selectListing_host(conn)

Please input host name: Celine
For this popular host,you can know that the listing he/she owned is Rental unit in Hong Kong,with a Entire home/apt type and hold on HK$2336.


Query for template tid:6

Find the most expensive room name

In [ ]:

def listMostExpensiveRoomName(conn):

    with conn:
            sql = "SELECT TEXT FROM TEMPLATE WHERE TID= 6"
            cur = conn.cursor()
            cur.execute(sql)
            result = cur.fetchone()
            template = result[0]
            #print(template)

            sql = """
            SELECT A.name, L.neighbourhood, A.price
            FROM Airbnb A
            JOIN Location L ON A.id = L.id
            ORDER BY A.price DESC
            LIMIT 1 """
            cur = conn.cursor()
            cur.execute(sql)
            result_set = cur.fetchall()
            if result_set:
                for result in result_set:
                  name = result[0]
                  neighbourhood = result[1]
                  price = result[2]
                  print(template.replace("{name}",str(result[0])).replace("{neighbourhood}",str(result[1])).replace("{price}",str(result[2])))
            else:
                print ("NO such room name")


In [ ]:
listMostExpensiveRoomName(conn)

The Most expensive room name is Rental unit in Hong Kong, the district is nearby Central & Western, the price is HK$150000.


Query for template tid:7

List recently reviews romm name and the number of reviews per month

In [ ]:
def listRecentlyReviewsRoomName(conn):

    line = input("Please input id: ")
    with conn:
            sql = "SELECT TEXT FROM TEMPLATE WHERE TID= 7"
            cur = conn.cursor()
            cur.execute(sql)
            result = cur.fetchone()
            template = result[0]
            #print(template)

            sql = """

               SELECT A.NAME,R.REVIEWS_PER_MONTH
               FROM Airbnb A
               JOIN Review R ON A.id = R.id
               WHERE A.id=?
               ;
               """
            cur = conn.cursor()
            cur.execute(sql,(line,))
            result = cur.fetchone()
            if result is None:
             print("No such record")
            else:

                print(template.replace("{name}",str(result[0])).replace("{reviews_per_month}",str(result[1])))


In [ ]:
database = dataPath + 'Airbnb.db'
conn = create_connection(database)
listRecentlyReviewsRoomName(conn)

Please input id: 17891
The listing Rental unit in Hong Kong Island has just received the reviews, and it got the average 0.45 item of customer comments per month.


Search by price range

In [ ]:
def selectCertainlisting(conn):
    """
    Given the price range, select the listings filtered by such condition.
    """

    line = input("Please input lowest_price, highest_price:")
    values = line.split(',')
    for i in range(0, len(values)):
        values[i] = values[i].strip()

    with conn:


        sql = "SELECT * FROM AIRBNB WHERE PRICE BETWEEN ? AND ? "
        cur = conn.cursor()
        cur.execute(sql, (values[0], values[1]))
        result_set = cur.fetchall()
        for result in result_set:
            printRoomInfo(result[0], conn)
            print("=================================================")
        print("Total", len(result_set), "choice(s).")

## Inserts new data
Then we insert 5 tuples to Airbnb table.

In [ ]:
database = dataPath + 'Airbnb.db'
conn = create_connection(database)
all_values = [('123', 'Rental unit in Victoria harbour', 'Entire home/apt', 10, 10000),
    ('124', 'Rental unit in Wan Chai', 'Private room ', 7, 2000),
    ('125', 'Rental unit in Prince Edward', 'Hotel room', 15, 1500),
    ('126', 'Rental unit in Mong Kok ', 'Shared room', 8, 1000),
    ('127', 'Rental unit in Hong Kong', 'Private room', 12, 900)]
for values in all_values:
    addAirbnb(conn, values)

##List all room

In [ ]:
def listALLRooms(conn):
    """
    List all Airbnb in the database.
    """
    print("All Airbnb in the database now:")
    with conn:
        sql = "SELECT ID FROM AIRBNB"
        cur = conn.cursor()
        cur.execute(sql)
        result_set = cur.fetchall() # store the result.
        for result in result_set: # show the result in to different row.
            print(result[0])
        print("Total", len(result_set), "Airbnb(s).") # len here is how many result rows.

In [ ]:
database = dataPath + 'Airbnb.db'
conn = create_connection(database)
listALLRooms(conn)

## Delete data

In [ ]:
def deleteRoom(conn):
    """
    Delete a flight from database by id
    """
    listALLRooms(conn)
    line = input("Please input the id to delete: ")
    line = line.strip()
    print(line)
    with conn:
        sql = "DELETE * FROM AIRBNB WHERE ID = ?"
        cur = conn.cursor()
        cur.execute(sql, [line])
        conn.commit()
        print("succeed to delete room ", line)

#ALL together

ALL together test

In [ ]:
import sqlite3
from sqlite3 import Error

def showOptions():
    """ Show the options """
    choices = [ "close the system","print room information (by id)", "list the most popular room type", "list the most expensive room name",
			"select certain listing (by lowest_price,highest_price )","print average minimum night of certain room type","select listing by host", "delete a room (by id)",
			"listRecentlyReviewsRoomName","search room type and price by id","check host and neighbourhood by id" ,"exit"]
    print("---------------------------")
    print("Please choose following option:")
    print("0. ", choices[0])
    print("1. ", choices[1])
    print("2. ", choices[2])
    print("3. ", choices[3])
    print("4. ", choices[4])
    print("5. ", choices[5])
    print("6. ", choices[6])
    print("7. ", choices[7])
    print("8. ", choices[8])
    print("9. ", choices[9])
    print("10. ", choices[10])


def listALLRooms(conn):
    """
    List all Airbnb in the database.
    """
    print("All Airbnb in the database now:")
    with conn:
        sql = "SELECT ID FROM AIRBNB"
        cur = conn.cursor()
        cur.execute(sql)
        result_set = cur.fetchall() # store the result.
        for result in result_set: # show the result in to different row.
            id = result[0]
            id
        print("Total", len(result_set), "Airbnb(s).") # len here is how many result rows.


def printRoomInfo(id, conn):
    """
    Print out the infomation of a room given a id
    """
    with conn:
        sql = "SELECT * FROM AIRBNB WHERE ID = ?"
        cur = conn.cursor()
        cur.execute(sql, [id])
        result = cur.fetchone()
        if result is None:
            print("No such rooms")
        else:
            headers = ["id", "name", "room_type", "minimum_night", "price"]
            for i in range(0, 5):
                print(headers[i], ":", result[i])
def printRoomByNo(conn):
    """
    Select out a room according to the id.
    """
    listALLRooms(conn)
    line = input("Please input the id to print info:")
    line = line.strip()
    printRoomInfo(line, conn)



def listMost_PopularRoomType(conn):

    with conn:
            sql = "SELECT TEXT FROM TEMPLATE WHERE TID= 1"
            cur = conn.cursor()
            cur.execute(sql)
            result = cur.fetchone()
            template = result[0]
            # print(template)

            sql = """
                SELECT NAME,MAX(NUMBER_OF_REVIEWS) AS MAX_NUMBER_REVIEWS,A.ID
                FROM Airbnb AS A,REVIEW AS R
                WHERE A.ID = R.ID
                  """
            cur = conn.cursor()
            cur.execute(sql)
            result_set = cur.fetchall()
            if result_set:
                for result in result_set:

                  print(template.replace("{name}",str(result[0])).replace("{number_of_reviews}",str(result[1])).replace("{id}",str(result[2])))
            else:
                print ("NO such room type")




def listMostExpensiveRoomName(conn):

    with conn:
            sql = "SELECT TEXT FROM TEMPLATE WHERE TID= 6"
            cur = conn.cursor()
            cur.execute(sql)
            result = cur.fetchone()
            template = result[0]
            #print(template)

            sql = """
            SELECT A.name, L.neighbourhood, A.price
            FROM Airbnb A
            JOIN Location L ON A.id = L.id
            ORDER BY A.price DESC
            LIMIT 1 """
            cur = conn.cursor()
            cur.execute(sql)
            result_set = cur.fetchall()
            if result_set:
                for result in result_set:
                  name = result[0]
                  neighbourhood = result[1]
                  price = result[2]
                  print(template.replace("{name}",str(result[0])).replace("{neighbourhood}",str(result[1])).replace("{price}",str(result[2])))
            else:
                print ("NO such room name")



def selectCertainlisting(conn):
    """
    Given the price range, select the listings filtered by such condition.
    """

    line = input("Please input lowest_price, highest_price:")
    values = line.split(',')
    for i in range(0, len(values)):
        values[i] = values[i].strip()

    with conn:


        sql = "SELECT * FROM AIRBNB WHERE PRICE BETWEEN ? AND ? "
        cur = conn.cursor()
        cur.execute(sql, (values[0], values[1]))
        result_set = cur.fetchall()
        for result in result_set:
            printRoomInfo(result[0], conn)
            print("=================================================")
        print("Total", len(result_set), "choice(s).")



def printMinimumNight(conn):
    """
    Print out the average minimum night of certain room type
    """
    line=input("please print the room type:")
    with conn:
        sql = "SELECT TEXT FROM TEMPLATE WHERE TID= 4"
        cur = conn.cursor()
        cur.execute(sql)
        result = cur.fetchone()
        template = result[0]
        #print(template)

        sql = "SELECT FLOOR(AVG(MINIMUN_NIGHT)) AS AVERAGE_NIGHT FROM AIRBNB WHERE ROOM_TYPE =?"
        cur = conn.cursor()
        cur.execute(sql, (line,))
        result = cur.fetchone()
        if result is None:
            print("No such record")
        else:
            print(template.replace("{avg(minimum_night)}",str(result[0])))



def selectListing_host(conn):
    """
    Given the hostname(like some popular hosts), select all the listings filtered by this condition.
    """
    line = input("Please input host name: ")


    with conn:
        sql = "SELECT TEXT FROM TEMPLATE WHERE TID= 5"
        cur = conn.cursor()
        cur.execute(sql)
        result = cur.fetchone()
        template = result[0]
        #print(template)

        sql = "SELECT NAME,ROOM_TYPE,PRICE FROM AIRBNB WHERE ID=(SELECT ID FROM HOST WHERE HOST_NAME =?) "
        cur = conn.cursor()
        cur.execute(sql, (line,))
        result = cur.fetchone()
        if result is None:
            print("No such rooms")
        else:
            print(template.replace("{name}",str(result[0])).replace("{room_type}",str(result[1])).replace("{price}",str(result[2])))



def deleteRoom(conn):
    """
    Delete a flight from database by id
    """
    listALLRooms(conn)
    line = input("Please input the id to delete: ")
    line = line.strip()
    print(line)
    with conn:
        sql = "DELETE * FROM AIRBNB WHERE ID = ?"
        cur = conn.cursor()
        cur.execute(sql, [line])
        conn.commit()
        print("succeed to delete room ", line)



def listRecentlyReviewsRoomName(conn):

    line = input("Please input id: ")
    with conn:
            sql = "SELECT TEXT FROM TEMPLATE WHERE TID= 7"
            cur = conn.cursor()
            cur.execute(sql)
            result = cur.fetchone()
            template = result[0]
            #print(template)

            sql = """

               SELECT A.NAME,R.REVIEWS_PER_MONTH
               FROM Airbnb A
               JOIN Review R ON A.id = R.id
               WHERE A.id=?
               ;
               """
            cur = conn.cursor()
            cur.execute(sql,(line,))
            result = cur.fetchone()
            if result is None:
             print("No such record")
            else:

                print(template.replace("{name}",str(result[0])).replace("{reviews_per_month}",str(result[1])))

def printPrice_Roomtype(conn):
    """
    Print out the infomation of a listing's room type and price
    """
    line=input("please print the id:")
    with conn:
        sql = "SELECT TEXT FROM TEMPLATE WHERE TID= 2"
        cur = conn.cursor()
        cur.execute(sql)
        result = cur.fetchone()
        template = result[0]
        # print(template)

        sql = "SELECT PRICE,ROOM_TYPE FROM AIRBNB WHERE ID= ?"
        cur = conn.cursor()
        cur.execute(sql,(line,))
        result = cur.fetchone()

        if result is None:
            print("No such rooms")
        else:
            print(template.replace("{price}",str(result[0])).replace("{room_type}",str(result[1])))

def printNeighbour_host(conn):
    """
    Print out the infomation of a room given a neighbourhood
    """
    line=input("please print the id:")
    with conn:
        sql = "SELECT TEXT FROM TEMPLATE WHERE TID= 3"
        cur = conn.cursor()
        cur.execute(sql)
        result = cur.fetchone()
        template = result[0]
        #print(template)

        sql = "SELECT HOST_NAME,NEIGHBOURHOOD FROM HOST AS H,LOCATION AS L WHERE H.ID = L.ID AND H.ID= ?"
        cur = conn.cursor()
        cur.execute(sql,(line,))
        result = cur.fetchone()
        if result is None:
            print("No such rooms")
        else:
            print(template.replace("{host_name}",str(result[0])).replace("{neighbourhood}",str(result[1])))




"""main"""
dataPath = '/content/drive/MyDrive/Colab Notebooks/data/'
database = dataPath + 'Airbnb.db'
conn = create_connection(database)
while True:
    showOptions()
    choice = input('Please enter your option: ')
    # with conn:
    if choice == '0':
        conn.close()
        break
    elif choice == '1':
        printRoomByNo(conn)
    elif choice == '2':
        listMost_PopularRoomType(conn)
    elif choice == '3':
        listMostExpensiveRoomName(conn)
    elif choice == '4':
        selectCertainlisting(conn)
    elif choice == '5':
        printMinimumNight(conn)
    elif choice == '6':
        selectListing_host(conn)
    elif choice == '7':
        deleteRoom(conn)
    elif choice == '8':
        listRecentlyReviewsRoomName(conn)
    elif choice == '9':
        printPrice_Roomtype(conn)
    elif choice == '10':
        printNeighbour_host(conn)
    else:
        print("This option is not available")
        continue



---------------------------
Please choose following option:
0.  close the system
1.  print room information (by id)
2.  list the most popular room type
3.  list the most expensive room name
4.  select certain listing (by lowest_price,highest_price )
5.  print average minimum night of certain room type
6.  select listing by host
7.  delete a room (by id)
8.  listRecentlyReviewsRoomName
9.  search room type and price by id
10.  check host and neighbourhood by id
Please enter your option: 1
All Airbnb in the database now:
Total 6422 Airbnb(s).
